<a href="https://colab.research.google.com/github/daman1209arora/cnn-mnist/blob/main/Combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports 

from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
from torch import nn, optim
import torch
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torchvision


In [2]:
import torch
import torchvision 

### All the parameters and hyperparameters

In [3]:
data = './data'
mnist_test = './data/mnist_test.csv'
mnist_train = './data/mnist_train.csv'

train_batch_size = 64
test_batch_size = 128
epochs = 5
alpha = 0.01
momentum = 0.09

In [4]:
random_seed = 1
torch.backends.cudnn.enabled = False 
torch.manual_seed(random_seed)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
from torch.utils.data import Dataset

In [7]:
# custom dataset
class MNISTDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X.iloc[i, :]
        data = np.asarray(data).astype(np.uint8).reshape(28, 28, 1)
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

In [8]:
import pandas as pd
import numpy as np

In [9]:
def get_manual_data():
  # read the data
  df_train = pd.read_csv(mnist_train)
  df_test = pd.read_csv(mnist_test)
  # get the image pixel values and labels
  train_labels = df_train.iloc[:, 0]
  train_images = df_train.iloc[:, 1:]
  test_labels = df_test.iloc[:, 0]
  test_images = df_test.iloc[:, 1:]
  return (train_labels, train_images, test_labels, test_images)

In [10]:
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, ToPILImage
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

In [11]:
def dataset_loader(custom = False , data_transform = None):
  if custom:
    train_labels, train_images, test_labels, test_images = get_manual_data()

    train_data = MNISTDataset(train_images, train_labels, data_transform)
    test_data = MNISTDataset(test_images, test_labels, data_transform)
    # dataloaders
    train_loader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=test_batch_size, shuffle=True)
  
  else:
    mnist = MNIST(download=True, train=True, root="./data").train_data.float()
    data_transform1 = Compose([ Resize((224, 224)),ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])
    # data_transform1 = Compose([ ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])
    train_loader = DataLoader(MNIST(download=True, root="./data", transform=data_transform1, train=True),
                              batch_size=train_batch_size, shuffle=True)

    test_loader = DataLoader(MNIST(download=False, root="./data", transform=data_transform1, train=False),
                            batch_size=test_batch_size, shuffle=False)
    
  return train_loader, test_loader



Models

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [13]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size =5, stride = (1,1))
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size =5, stride=(1,1))
    self.pool = nn.MaxPool2d(kernel_size =2, stride=2)
    self.fc1 = nn.Linear(in_features = 16*5*5, out_features=120)
    self.fc2 = nn.Linear(in_features = 120, out_features = 84)
    self.fc3 = nn.Linear(in_features=84,out_features=10)

  def forward(self,x):
    layer1 = self.pool(F.relu(self.conv1(x)))
    layer2 = self.pool(F.relu(self.conv2(layer1)))
    one_hot = layer2.view(-1,400)
    layer3 = F.relu(self.fc1(one_hot))
    layer4 = F.relu(self.fc2(layer3))
    out = F.softmax(self.fc3(layer4),dim=1) 
    return out

In [14]:
class ConvNet2(nn.Module):
  def __init__(self):
    super(ConvNet2,self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 24, kernel_size =5, stride = (1,1),padding=(2,2))
    self.conv2 = nn.Conv2d(in_channels = 24, out_channels = 48, kernel_size =5, stride=(1,1),padding=(2,2))
    self.conv3 = nn.Conv2d(in_channels = 48, out_channels = 64, kernel_size =5, stride=(1,1),padding=(2,2))
    self.pool = nn.MaxPool2d(kernel_size =2, stride=2)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    self.fc1 = nn.Linear(64*16,256)
    self.fc2 = nn.Linear(256,10)

  def forward(self,x):
    layer1 = self.pool(F.relu(self.conv1(x)))
    layer2 = self.pool(F.relu(self.conv2(layer1)))
    layer3 = self.pool2(F.relu(self.conv3(layer2)))
    one_hot = layer3.view(-1,64*16)
    layer4 = F.relu(self.fc1(one_hot))
    out = F.softmax(self.fc2(layer4),dim=1) 
    return out

In [15]:
# 1 -> custom 
# 2 -> resnet18 
# 3 -> vgg16
# 4 -> resnet54
def model_choose(val):
  if val == 1:
    return ConvNet2()
  elif val == 2:
    x =models.resnet18(pretrained=True)
    for param in x.parameters():
      param.requires_grad = False
    x.fc = nn.Linear(512, 10)
    return x
  elif val == 3:
    x = models.vgg16(pretrained = True)
    for param in x.parameters():
      param.requires_grad = False
    x.classifier[6] = nn.Linear(4096,10)
    return x
  elif val == 4:
    x =models.resnet34(pretrained=True)
    for param in x.parameters():
      param.requires_grad = False
    x.fc = nn.Linear(512, 10)
    return x
  else:
    print("error")
    return None   

In [23]:
model = model_choose(2).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.05)

data_transform = Compose([ToPILImage(),  Resize((224, 224)),ToTensor(), Normalize((0.5, ), (0.5, ))])
# data_transform = Compose([Resize((32,32)), ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])

In [17]:
train_loader, val_loader = dataset_loader(False,data_transform)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw





Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [18]:
#imports 
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time


In [19]:
def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    # just an utility printing function
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")


In [26]:
def run():  
  model.to(device)

  start_ts = time.time()

  losses = []
  batches = len(train_loader)
  val_batches = len(val_loader)

  # loop for every epoch (training + evaluation)
  for epoch in range(epochs):
      total_loss = 0

      # progress bar (works in Jupyter notebook too!)
      progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

      # ----------------- TRAINING  -------------------- 
      # set model to training
      
      
      for i, data in progress:
          X, y = data[0].to(device), data[1].to(device)
          X = torch.cat([X,X,X],dim=1)
          X = X.to(device)
          # training step for single batch
          model.zero_grad()
          outputs = model(X)
          loss = loss_function(outputs, y)
          loss.backward()
          optimizer.step()

          # getting training quality data
          current_loss = loss.item()
          total_loss += current_loss

          # updating progress bar
          progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
          
      # releasing unceseccary memory in GPU
      if torch.cuda.is_available():
          torch.cuda.empty_cache()
      
      # ----------------- VALIDATION  ----------------- 
      val_losses = 0
      precision, recall, f1, accuracy = [], [], [], []
      
      # set model to evaluating (testing)
      model.eval()
      with torch.no_grad():
          for i, data in enumerate(val_loader):
              X, y = data[0].to(device), data[1].to(device)
              X = torch.cat([X,X,X],dim=1)
              X = X.to(device)
              outputs = model(X) # this get's the prediction from the network

              val_losses += loss_function(outputs, y)

              predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
              
              # calculate P/R/F1/A metrics for batch
              for acc, metric in zip((precision, recall, f1, accuracy), 
                                    (precision_score, recall_score, f1_score, accuracy_score)):
                  acc.append(
                      calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                  )
            
      print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
      print_scores(precision, recall, f1, accuracy, val_batches)
      losses.append(total_loss/batches) # for plotting learning curve
  print(f"Training time: {time.time()-start_ts}s")

In [28]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
train_batch_size = 128
test_batch_size = 128
print("resnet 18 128x128")
run()

model = model_choose(3).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
train_batch_size = 64
print("vgg16 0.01")
run()

optimizer = torch.optim.Adam(model.parameters(),lr=0.02)
print("vgg16 0.02")
run()

optimizer = torch.optim.Adam(model.parameters(),lr=0.05)
print("vgg16 0.05")
run()

optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
train_batch_size = 128
test_batch_size = 128
print("vgg16 128x128")
run()

model = model_choose(4).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
train_batch_size = 64
print("resnet34 0.01")
run()

optimizer = torch.optim.Adam(model.parameters(),lr=0.02)
print("resnet 34 0.02")
run()

optimizer = torch.optim.Adam(model.parameters(),lr=0.05)
print("resnet 0.05")
run()

optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
train_batch_size = 128
test_batch_size = 128
print("resnet 128x128")
run()
print("done")

resnet 18 128x128



Epoch 1/5, training loss: 0.3616415075038654, validation loss: 0.48760876059532166
	     precision: 0.9685
	        recall: 0.9685
	            F1: 0.9669
	      accuracy: 0.9685



Epoch 2/5, training loss: 0.30499968720036674, validation loss: 0.5372755527496338
	     precision: 0.9650
	        recall: 0.9646
	            F1: 0.9624
	      accuracy: 0.9640



Epoch 3/5, training loss: 0.2604337577318541, validation loss: 0.4704928398132324
	     precision: 0.9670
	        recall: 0.9656
	            F1: 0.9646
	      accuracy: 0.9657



Epoch 4/5, training loss: 0.2514881468088252, validation loss: 0.46719902753829956
	     precision: 0.9659
	        recall: 0.9648
	            F1: 0.9635
	      accuracy: 0.9649


...


Epoch 1/5, training loss: 0.5146595473806006, validation loss: 0.7848599553108215
	     precision: 0.9566
	        recall: 0.9561
	            F1: 0.9543
	      accuracy: 0.9556



Epoch 2/5, training loss: 0.4265988745130613, validation loss: 0.7384164333343506
	     precision: 0.9588
	        recall: 0.9573
	            F1: 0.9559
	      accuracy: 0.9574



Epoch 3/5, training loss: 0.4000405404440898, validation loss: 0.7460170388221741
	     precision: 0.9578
	        recall: 0.9562
	            F1: 0.9548
	      accuracy: 0.9561



Epoch 4/5, training loss: 0.369221431241788, validation loss: 0.7336481213569641
	     precision: 0.9570
	        recall: 0.9566
	            F1: 0.9548
	      accuracy: 0.9561



Epoch 5/5, training loss: 0.3523805261261043, validation loss: 0.8559110164642334
	     precision: 0.9477
	        recall: 0.9458
	            F1: 0.9431
	      accuracy: 0.9452
Training time: 1828.613564491272s
done
